# Lion-Q-VQE
## Our teams take on implmenting a VQE and using it to optimize a Financial Portfolio

### Imports

In [37]:
from qiskit.circuit.library import TwoLocal, ZZFeatureMap, EfficientSU2
from qiskit.result import QuasiDistribution
from qiskit.primitives import Sampler as sampler_sim
from qiskit.primitives import BackendSampler, Estimator

from qiskit_algorithms.gradients import FiniteDiffEstimatorGradient, SPSAEstimatorGradient, LinCombEstimatorGradient, ParamShiftEstimatorGradient, ParamShiftSamplerGradient, QFI, DerivativeType, LinCombQGT
from qiskit_optimization.translators import from_docplex_mp
from qiskit_algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer

import numpy as np
import matplotlib.pyplot as plt
import datetime

### Global Variables

In [29]:
# set number of assets which is equal to number of qubits
num_assets = 25
seed = 479

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2023, 1, 1),
    end=datetime.datetime(2023, 1, 30),
    seed=seed,
)

print(data)

In [30]:
data.run()
ev = data.get_period_return_mean_vector() # Initial state to optimize over
covariance = data.get_period_return_covariance_matrix() # Hamiltonian Matrix

risk_factor = 0.5  # set risk factor
budget = 20  # set budget
#penalty = num_assets  # set parameter to scale the budget penalty term

qp = PortfolioOptimization(expected_returns=ev, covariances=covariance, risk_factor=risk_factor, budget=budget).to_quadratic_program()
print(data)

In [31]:
estimator = Estimator()
optimizer = COBYLA()

### Ansatz Implementation Comparison and Analysis

In [42]:
# 3 common quantum circuits to use for ansatz with linear entanglement
ZZ = ZZFeatureMap(feature_dimension=num_assets, entanglement='linear', reps=1)
SU2 = EfficientSU2(num_assets, entanglement='linear', reps=1)
TwoL = TwoLocal(num_assets, entanglement='linear', reps=1)
linear_ansatz = [ZZ, SU2, TwoL]

# 3 common quantum circuits to use for ansatz with cyclic entanglement
ZZ = ZZFeatureMap(feature_dimension=num_assets, entanglement='cyclic', reps=1)
SU2 = EfficientSU2(num_assets, entanglement='cyclic', reps=1)
TwoL = TwoLocal(num_assets, entanglement='cyclic', reps=1)
cyclic_ansatz = [ZZ, SU2, TwoL]

# 3 common quantum circuits to use for ansatz with full entanglement
ZZ = ZZFeatureMap(feature_dimension=num_assets, entanglement='full', reps=1)
SU2 = EfficientSU2(num_assets, entanglement='full', reps=1)
TwoL = TwoLocal(num_assets, entanglement='full', reps=1)
full_ansatz = [ZZ, SU2, TwoL]

# master list for ansatz circuits shape is 3 x 3
ansatz_list = [linear_ansatz, cyclic_ansatz, full_ansatz]

### Gradient Comparison and Analysis

In [21]:
grad_finite = FiniteDiffEstimatorGradient(estimator, epsilon=0.001)
grad_SPSA = SPSAEstimatorGradient(estimator, epsilon=0.001, batch_size=10, seed=50)
qgt = LinCombQGT(estimator, derivative_type=DerivativeType.COMPLEX)
grad_pshift = ParamShiftEstimatorGradient(estimator)
pshift_sampler = ParamShiftSamplerGradient(sampler_sim())
grad_natural = QFI(qgt)

In [ ]:
def gd(init_params, p, iterations, H, step):
    params = [[init_params], [init_params], [init_params], [init_params]]
    qc = EfficientSU2(num_qubits=4, reps=p, entanglement="linear", insert_barriers = True)
    for i in range(iterations):
        grad_finite_res = grad_finite.run([qc], [H], [params[0][i]]).result().gradients[0]
        params[0].append(params[0][i] - step * grad_finite_res)
        
        grad_spsa_res = grad_SPSA.run([qc], [H], [params[1][i]]).result().gradients[0]
        params[1].append(params[1][i] - step * grad_spsa_res)
        
        qfis = grad_natural.run([qc], [params[2][i]]).result().qfis[0]
        tmp_grad = grad_finite.run([qc], [H], [params[2][i]]).result().gradients[0]
        params[2].append(params[2][i] - step * np.dot(np.linalg.pinv(qfis), tmp_grad))

        grad_pshift_res = grad_pshift.run([qc], [H], [params[3][i]]).result().gradients[0]
        params[3].append(params[3][i] - step * grad_pshift_res)
        
        #step = step * .75 # Modification to decay the step distance 
    return params

### Entanglement Schemes Comparison and Analysis